# **Проект исследования вакансий Junior аналитиков с сайта LinkedIn**

 **$\color{blue}{\text{КОММЕНТАРИЙ}}$**

Привет, Евгений Меня зовут Катя и я проведу ревью твоей работы

## **Цель исследования**

Визуализировать информацию о рынке вакансий для аналитиков в Европе, в разрезе:
- по стране и по типу занятости
- общее количество вакансий
- нанимающих компаний с указанием количества вакансий,
- ТОП 10 сфер деятельности компаний, которые нанимают аналитиков
- размер компаний и количество вакансий
- требуемые хард скилы
- Дополнительно (необязательно) - зависимость количества кандидатов на вакансию от даты публикации


### **Описание  данных:**

Предоставлен датасет с сырыми данными после парсинга сайта LinkedIn, который необходимо обработать и создать датасет, содержащий:
- наименование вакансии
- город
- страна
- тип занятости (online, hybride, on-site)
- компания
- размер компании (количество работников)
- сфера деятельности компании
- требуемые хард скилы
- дата публикации вакансии
- количество кандидатов на вакансию

### **План исследования**

Работать будем в следующем порядке:
- импорт данных
- детальный анализ одной страницы сайта из файла, а также непосредственно на сайте
- поиск тегов и классов для выделения требуемой информации
- выделение требуемой информации в отдельные столбцы
- выделение информации из столбцов
- удаление дубликатов и ненужных столбцов

- создание дашборда

 **$\color{blue}{\text{КОММЕНТАРИЙ}}$**

Очень классно что ты оформляешь проект как надо - с названием, описениаем данных, целью, шагами исследования

In [ ]:
# перед началом работы импортируем библиотеки, которые могут понадобиться в работе над проектом

import pandas as pd
import math
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import scipy.stats as stats
from urllib.parse import urlencode 
import re
import requests
from calendar import day_name
import plotly
import plotly.graph_objects as go
import plotly.express as px

%matplotlib inline

In [ ]:
from bs4 import BeautifulSoup
import html5lib

In [ ]:
# установим неограниченную ширину колонок датафреймов
pd.set_option('display.max_colwidth', None)

## **Импорт данных**

In [ ]:
# используя конструкцию try-ecxept для возможности открыть локально и с внешнего источника
# для загрузки сразу в датафрейм
try:
    raw_df = pd.read_csv(r'C:\Users\ezhmui04\Documents\Поиск работы\Тестовые задания\masterskaya_parsing_LinkedIn_2023_05_23.csv')
    
except:

    # используем api
    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
    public_key = 'https://disk.yandex.ru/d/gsL7aCpUvlysoQ'
    # получаем url
    final_url = base_url + urlencode(dict(public_key=public_key))
    response = requests.get(final_url)
    download_url = response.json()['href']
    # загружаем файл в df
    download_response = requests.get(download_url)
    raw_df = pd.read_csv(download_url)

 **$\color{blue}{\text{КОММЕНТАРИЙ}}$**

Здорово Кстати с гугл диска гораздо проще читать файлы

In [ ]:
# посмотрим на одну строку датафрейма
raw_df.head(1)

Unnamed: 0  \
0           0   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [ ]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 998 entries, 0 to 997
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  998 non-null    int64 
 1   html        998 non-null    object
dtypes: int64(1), object(1)
memory usage: 15.7+ KB


У нас есть спарсенные данные в датафрейме. В нем 998 строк, в каждой есть номер и  страница с описанием вакансии в виде текста (формат html). Необходимо из данных в таком виде получить их в удобном для дальнейшей визуализации виде.

 **$\color{blue}{\text{КОММЕНТАРИЙ}}$**

Великолепно

## **Парсинг требуемых данных**

### **Рассмотрение одного блока с вакансией на странице**

In [ ]:
from IPython.display import display, HTML

In [ ]:
# запишем в переменную soup_test распарсенное значение одной страницы, что бы полее подробно его изучить
soup_test = BeautifulSoup(raw_df.iloc[0]['html'], 'lxml')

In [ ]:
# и посмотрим на него в структурированном виде
print(soup_test.prettify())

<html>
 <body>
  <div>
   <div class="jobs-details__main-content jobs-details__main-content--single-pane full-width">
    <!-- -->
    <div>
     <div class="jobs-unified-top-card t-14">
      <!-- -->
      <div class="relative jobs-unified-top-card__container--two-pane">
       <div class="jobs-unified-top-card__content--two-pane">
        <!-- -->
        <a class="ember-view" href="/jobs/view/3609065367/?alternateChannel=search&amp;refId=AxoCJAz0Vnn3xzCIlNO7ng%3D%3D&amp;trackingId=mdZWGb64G6DZP8qCAPQ%2BmQ%3D%3D&amp;trk=d_flagship3_search_srp_jobs" id="ember425">
         <h2 class="t-24 t-bold jobs-unified-top-card__job-title">
          Data Analyst
         </h2>
        </a>
        <div class="jobs-unified-top-card__primary-description">
         <span class="jobs-unified-top-card__subtitle-primary-grouping t-black">
          <span class="jobs-unified-top-card__company-name">
           <a class="ember-view t-black t-normal" href="/company/pharmiweb-jobs/life/" id="ember426">


In [ ]:
# и посмотрим на него в виде страницы
display(HTML(raw_df['html'][45]))

In [ ]:
# проверим тег и класс для выделения названия вакансии
soup_test.find_all('h2', class_="t-24 t-bold jobs-unified-top-card__job-title")

[<h2 class="t-24 t-bold jobs-unified-top-card__job-title">Data Analyst</h2>]

In [ ]:
soup_test.find('h2', class_="t-24 t-bold jobs-unified-top-card__job-title").text.strip()

'Data Analyst'

In [ ]:
# проверим тег и класс для выделения типа занятости
soup_test.find('span', class_="jobs-unified-top-card__workplace-type").text.strip()

'On-site'

In [ ]:
# ТЕСТОВЫЕ ИЗВЛЕЧЕНИЯ, В ДАЛЬНЕЙШЕМ НЕ ИСПОЛЬЗУЮТСЯ

# извлечение инфо из одного тега
# Извлечение названия вакансии
job_title = soup_test.find('strong', string='Job Title:').next_sibling.strip()
print('Название вакансии:', job_title)

# Извлечение местоположения
location = soup_test.find('strong', string='Location:').next_sibling.strip()
print('Местоположение:', location)

# Извлечение тип работы
job_type = soup_test.find('strong', string='Job Type:').next_sibling.strip()
print('Тип контракта:', job_type)


Название вакансии: Data Analyst
Местоположение: Basel, Switzerland
Тип контракта: Contract


In [ ]:
# проверим тег и класс для выделения названия компании
soup_test.find('span', class_ = 'jobs-unified-top-card__company-name').text.strip()

'PharmiWeb.Jobs: Global Life Science Jobs'

In [ ]:
# проверим тег и класс для выделения местоположения компании
soup_test.find('span', class_ = 'jobs-unified-top-card__bullet').text.strip()

'Basel, Basel, Switzerland'

In [ ]:
# проверим тег и класс для выделения количества персонала в компании
soup_test.find('span', class_ = 'jobs-company__inline-information').text.strip()

'11-50 employees'

In [ ]:
# проверим тег и класс для выделения сферы деятельности компании
div_tag = soup_test.find('div', class_='t-14 mt5')
text = div_tag.next_element.strip()
print(text)

Staffing & Recruiting


In [ ]:
# Извлечение текстов "Full-time" и "Entry level"
li_tag = soup_test.find('li', class_='jobs-unified-top-card__job-insight')
text = li_tag.span.get_text(strip=True)
full_time, entry_level = text.split('·')
print('Full-time:', full_time.strip())
print('Entry level:', entry_level.strip())

Full-time: Full-time
Entry level: Entry level


In [ ]:
# проверим тег и класс для выделения информации о сроке публикации вакансии
soup_test.find('span', class_ = 'jobs-unified-top-card__posted-date').text.strip()

'1 week ago'

In [ ]:
# проверим тег и класс для выделения полного описания вакансии
text = soup_test.find(
    'div', class_='jobs-box__html-content jobs-description-content__text t-14 t-normal jobs-description-content__text--stretch').text

 **$\color{blue}{\text{КОММЕНТАРИЙ}}$**

Отлично что ты проверяешь корректность получаемой информации для одного элемента

### Функции выделения информации по тегу и классу

In [ ]:
# функция выделения двух скиллов из блока скиллов
def extract_skills(string, num_skills=2):
    # Извлекаем подстроку, содержащую все навыки
    match = re.search(r'Skills:(.*)', string)
    if not match:
        return None

    # Извлекаем все навыки с помощью регулярного выражения
    skills = re.findall(r'\w[\w\s]+\w', match.group(1))

    # Возвращаем указанное количество навыков
    return skills[:num_skills]

# Пример использования
string = "'Skills: Английский язык, Анализ данных, +8 more'"
skills = extract_skills(string, num_skills=2)
print(skills)

['Английский язык', 'Анализ данных']


 **$\color{blue}{\text{КОММЕНТАРИЙ}}$**

Просто великолепная функция, однако у нас основная часть хард скиллов содержится в тексте вакансии И именно оттуда нам нужно их доставать

In [ ]:
# основная функция для извлечения данных по тегу и классу
# используем конструкцию try-except для возврата "no info" для тех случаев, когда информации на странице нет
def info(x, tag, cl):
    try: res = BeautifulSoup(x).find(
    tag, class_=cl).text.strip()  
    except: res = 'no info'
        
    return res    

 **$\color{blue}{\text{КОММЕНТАРИЙ}}$**

ЗАМЕЧАТЕЛЬНО что ты создаешь функцию

In [ ]:
# функция для извлечения данных по тегу и строке
# используем конструкцию try-except для возврата "no info" для тех случаев, когда информации на странице нет
def info_str(x, tag, string):
    try: res = BeautifulSoup(x).find(tag, string=string).next_sibling.strip()
    except: res = 'no info'
        
    return res 

In [ ]:
# функция для извлечения данных по тегу и классу, но случаев когда информация в следующем элементе после тега
# используем конструкцию try-except для возврата "no info" для тех случаев, когда информации на странице нет
def comp_sphere(x, tag, cl):
    try: 
        div_tag = BeautifulSoup(x).find(tag, class_=cl)
        res = div_tag.next_element.strip()
    except: res = 'no info'
        
    return res

In [ ]:
# функции для извлечения данных по тегу и классу из блока информации о вакансии
# используем конструкцию try-except для возврата "no info" для тех случаев, когда информации на странице нет
def employment(x, tag, cl):
    try: 
        div_tag = BeautifulSoup(x).find(tag, class_=cl)
        text = li_tag.span.get_text(strip=True)
        full_time, entry_level = text.split('·')
        res = full_time
    except: res = 'no info'
        
    return res

def level(x, tag, cl):
    try: 
        div_tag = BeautifulSoup(x).find(tag, class_=cl)
        text = li_tag.span.get_text(strip=True)
        full_time, entry_level = text.split('·')
        res = entry_level
    except: res = 'no info'
        
    return res


 **$\color{blue}{\text{КОММЕНТАРИЙ}}$**

Все круто

### **Создание столбца с названием должности**

Видим, что название вакансии находится в блоке 
< h2 class="t-24 t-bold jobs-unified-top-card__job-title"> Data Analyst</h2 >

Сформируем новый столбец, в котором будет название вакансии

In [ ]:
df = raw_df.copy()

In [ ]:
# создадим столбец с названием вакансии
df['job_title'] = df['html'].apply(lambda x: BeautifulSoup(x).find(
    'h2', class_="t-24 t-bold jobs-unified-top-card__job-title").text.strip())

In [ ]:
df['job_title'].value_counts()

Data Analyst                                         93
Online Data Analyst                                  38
Data Analyst - Milano                                28
Data Analyst (m/w/d)                                 25
Datový analytik                                      19
                                                     ..
ESG Analyst - Real Estate (m/w/d)                     1
Projektmanager / Projektleiter Automotive (m/w/d)     1
Ingenieur Softwareentwicklung (m/w/d)                 1
Mitarbeiter Marketing (m/w/d)                         1
Analista de datos BI                                  1
Name: job_title, Length: 560, dtype: int64

In [ ]:
df['job_title'].unique()

array(['Data Analyst', 'Data Analyst - Logistics',
       'Data Analyst (Space & Planning)', 'Data Analyst - Hybrid Working',
       'Commercial Data Analyst', 'Asset Data Analyst',
       'Data Analyst - Hybrid', 'Data Analyst (FT)', 'Data Analyst H/F',
       'Data Analyst II', 'Data Analyst (9 Months FTC)',
       'Business Intelligence Analyst', 'Data Analyst (F/H)',
       'BI Analyst', 'Data Analyst Associate', 'Product Data Analyst',
       'Graduate Data Analyst', 'Digital Data Analyst',
       'Business Intelligence Analyst Junior', 'Data analyst H/F',
       'Data Analyst (m/f/d)', 'Analytics Consultant', 'Data Analyst F/H',
       'Consultor BI', 'Statistical Data Analyst',
       'Digital Data Analyst (H/F)',
       'Data Analyst - €60,- per hour - Amsterdam based',
       'Data & Analytics Senior Analyst',
       'JUNIOR DATA ANALYST – Aerospace', 'Data Analyst - Operations',
       'Data Analyst (H/F)', 'BUSINESS INTELLIGENCE ANALYST',
       'Business Intelligence Analys

Видим в списке вакансий 560 различных наименований, в том числе и вакансии, не являющиеся дата-аналитиками. Следующим этапом сократим количество наименований вакансий.

In [ ]:
# сначала приведем все к строчным буквам
df['job_title'] = df['job_title'].str.lower()

In [ ]:
# заменим видимые глазом синонимы на единое наименование, в том числе с иностранных языков
df['job_title'] = df['job_title'].replace('data-analist', 'data analyst')
df['job_title'] = df['job_title'].replace('analista de datos', 'data analyst')
df['job_title'] = df['job_title'].replace('analyste des donnees', 'data analyst')
df['job_title'] = df['job_title'].replace('analista de datos', 'data analyst')
df['job_title'] = df['job_title'].replace('datový analytik', 'data analyst')
df['job_title'] = df['job_title'].replace('analityk danych', 'data analyst')
df['job_title'] = df['job_title'].replace('datový analytik', 'data analyst')
df['job_title'] = df['job_title'].replace('dataanalytiker', 'data analyst')

In [ ]:
# оставим в датафрейме только вакансии с указанием data analyst или BI
df = df[(df['job_title'].str.contains('data analyst')) | (df['job_title'].str.contains('BI'))]

In [ ]:
df['job_title'].value_counts()

data analyst                           125
online data analyst                     38
data analyst - milano                   28
data analyst (m/w/d)                    26
reliability data analyst                19
                                      ... 
data analyst telco                       1
data analyst h/f h/f                     1
data analyst (m/w/d) im controlling      1
data analyst (mobile)                    1
data analyst job in overseas             1
Name: job_title, Length: 181, dtype: int64

Работодатели на LinkedIn не придерживаются единой философии именования вакансий, и, видимо, ориентированы не на тех, кто ищет вакансию по ее названию, а скорее на тех, кто ищет работу в конкретной компании.

 **$\color{blue}{\text{КОММЕНТАРИЙ}}$**

к сожалению это так Очень классная обработка и фильтрация данных по релевантности

### **Выделение кампании, и общего адреса**

In [ ]:
df['company'] = df['html'].apply(lambda x: BeautifulSoup(x).find(
    'span', class_="jobs-unified-top-card__company-name").text.strip())
df['adress'] = df['html'].apply(lambda x: BeautifulSoup(x).find(
    'span', class_="jobs-unified-top-card__bullet").text.strip())

### **Выделение размера компании**

In [ ]:
df['company_size'] = df['html'].apply(lambda x: info(x, 'span', 'jobs-company__inline-information'))

### **Выделение типа занятости**

In [ ]:
df['type_of_employment'] = df['html'].apply(lambda x: info(x, 'span', 'jobs-unified-top-card__workplace-type'))

### **Выделение сферы деятельности компании**

In [ ]:
df['company_sphere'] = df['html'].apply(lambda x: comp_sphere(x, 'div', 't-14 mt5'))

### **Выделение занятости**

In [ ]:
df['employment'] = df['html'].apply(lambda x: employment(
    x, 'li', 'jobs-unified-top-card__job-insight'))
df['level'] = df['html'].apply(lambda x: level(
    x, 'li', 'jobs-unified-top-card__job-insight'))

### **Выделение требуемых hard skills для вакансии**

In [ ]:
# в отдельную ячейку запишем весь текст вакансии
df['vac_text'] = df['html'].apply(lambda x: info(x,'div', 'jobs-box__html-content jobs-description-content__text t-14 t-normal jobs-description-content__text--stretch'))

In [ ]:
# и в отдельную те два скила, которые можно взять непосредственно со страницы
df['base_skills'] = df['html'].apply(lambda x: info(x,'span','visually-hidden'))

### **Выделение дата добавления вакансии**

In [ ]:
df['time_from_public'] = df['html'].apply(lambda x: info(
    x, 'span', 'jobs-unified-top-card__posted-date'))

### **Выделение количество кандидатов на вакансию**

In [ ]:
df['candidates'] = df['html'].apply(lambda x: info(
    x, 'span', 'jobs-unified-top-card__applicant-count'))

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 520 entries, 0 to 994
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          520 non-null    int64 
 1   html                520 non-null    object
 2   job_title           520 non-null    object
 3   company             520 non-null    object
 4   adress              520 non-null    object
 5   company_size        520 non-null    object
 6   type_of_employment  520 non-null    object
 7   company_sphere      520 non-null    object
 8   employment          520 non-null    object
 9   level               520 non-null    object
 10  vac_text            520 non-null    object
 11  base_skills         520 non-null    object
 12  time_from_public    520 non-null    object
 13  candidates          520 non-null    object
dtypes: int64(1), object(13)
memory usage: 60.9+ KB


### **Промежуточные итоги по парсингу**

На данном этапе мы получили датафрейм, содержащий в отдельных столбцах информацию о 
- названии должности
- компании
- адресе компании, с указанием города, региона и страны в одной ячейке
- размере компании
- сферы деятельности компании
- типу занятости на вакансию
- уровень на вакансию
- когда была опубликована вакансия относительно даты парсинга - 23.05.2023
- сколько кандидатов на вакансию
- текстовое описание вакансии

Далее необходимо:
- разделить столбец адреса на город и страну
- расчитать дату публикации вакансии
- выделить из описания вакансии hard skills

 **$\color{blue}{\text{КОММЕНТАРИЙ}}$**

Отлично Хочу отметить великолепное владение методами bs4

## **Выделение информации из столбцов**

In [ ]:
# уберем из датасета столбец с html страницей, она нам больше не понадобиться
df = df.drop('html', axis = 1)

 **$\color{blue}{\text{КОММЕНТАРИЙ}}$**

Здесь еще и Unnamed можно дропнуть

In [ ]:
df.sample(3)

Unnamed: 0                                                 job_title  \
298         298                                      data analyst (m/w/d)   
638         638                                     data analyst - milano   
425         425  senior data analyst (bangkok based, relocation provided)   

               company                                       adress  \
298  Charisma-Tec GmbH                                      Germany   
638          CPM Italy         San Donato Milanese, Lombardy, Italy   
425              Agoda  Düsseldorf, North Rhine-Westphalia, Germany   

               company_size type_of_employment     company_sphere  employment  \
298          2-10 employees            On-site    Human Resources  Full-time    
638        51-200 employees             Hybrid             Retail  Full-time    
425  5,001-10,000 employees            no info  Computer Software  Full-time    

            level  \
298   Entry level   
638   Entry level   
425   Entry level   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

### **Извлечение города и страны**

In [ ]:
df['adress'].unique()

array(['Basel, Basel, Switzerland', 'Coventry, England, United Kingdom',
       'South Molton, England, United Kingdom',
       'Lugano, Ticino, Switzerland',
       'Southampton, England, United Kingdom',
       'Leeds, England, United Kingdom',
       'Nuneaton, England, United Kingdom',
       'Paris, Île-de-France, France',
       'Cambridge, England, United Kingdom',
       'West Midlands, England, United Kingdom',
       'Schiphol, North Holland, Netherlands',
       'Chester, England, United Kingdom',
       'Craven Arms, England, United Kingdom',
       'Dublin, County Dublin, Ireland',
       'Belfast, Northern Ireland, United Kingdom',
       'Sunderland, England, United Kingdom',
       'Montévrain, Île-de-France, France',
       'Bristol, England, United Kingdom',
       'Solihull, England, United Kingdom',
       'Blackpool, England, United Kingdom',
       'Dijon, Bourgogne-Franche-Comté, France',
       'Manchester, England, United Kingdom',
       'Elliniko-Argyroupoli,

В адресах компаний видим, что есть группы из трех частей адреса: города, региона и страны, из двух и только из одного . Посмотрим, сколько разных вариантов есть.

In [ ]:
df['adr_len'] = df['adress'].apply(lambda x: len(x.split(',')))

In [ ]:
df['adr_len'].value_counts()

3    424
1     56
2     40
Name: adr_len, dtype: int64

Так как полностью заполненных данных большинство, и заполнить недостающие данные не представляется возможным, дальше будем действовать следующим образом: первая часть адреса будет городом, последняя - страной. У части вакансий эти город и страна будут совпадать.

In [ ]:
df['city'] = df['adress'].apply(lambda x: x.split(',')[0].strip())
df['country'] = df['adress'].apply(lambda x: x.split(',')[-1].strip())

In [ ]:
df['country'].unique()

array(['Switzerland', 'United Kingdom', 'France', 'Netherlands',
       'Ireland', 'Greece', 'Sweden', 'Lithuania', 'Belgium',
       'Luxembourg', 'Germany', 'Italy', 'Brussels Metropolitan Area',
       'Greater Paris Metropolitan Region', 'Spain', 'Hungary', 'Poland',
       'Czechia', 'Wroclaw Metropolitan Area', 'Portugal', 'Latvia',
       'Estonia', 'Greater Barcelona Metropolitan Area', 'Malta',
       'Austria', 'Cologne Bonn Region', 'Greater Munster Area',
       'Greater Banska Bystrica Area',
       'Greater Nuremberg Metropolitan Area', 'Berlin Metropolitan Area',
       'Romania', 'Norway', 'Rotterdam and The Hague', 'Denmark',
       'Bulgaria', 'Greater Pau Area', 'Monaco', 'Eindhoven Area',
       'Krakow Metropolitan Area'], dtype=object)

 **$\color{blue}{\text{КОММЕНТАРИЙ}}$**

Великолепно что ты вытащил страну Но здесь необходима небольшая обработка столбца - некоторые значения у нас не страны - например 'Brussels Metropolitan Area' и тд Работодатель не затруднился написать страну и сразу указал регио - бывает и такое

 **$\color{blue}{\text{КОММЕНТАРИЙ}}$**

Будет здорово если ты также проверишь корректность значений для городов

### **Извлечение даты публикации вакансии**

In [ ]:
df['time_from_public'].value_counts()

1 week ago        172
2 weeks ago       141
3 weeks ago        47
4 days ago         46
4 weeks ago        30
5 days ago         26
6 days ago         17
3 days ago         16
1 day ago          14
2 days ago          5
6 hours ago         3
9 hours ago         2
12 minutes ago      1
Name: time_from_public, dtype: int64

In [ ]:
df['time_from_public'] = df['time_from_public'].replace('1 week ago', 7)
df['time_from_public'] = df['time_from_public'].replace('2 weeks ago', 14)
df['time_from_public'] = df['time_from_public'].replace('3 weeks ago', 21)
df['time_from_public'] = df['time_from_public'].replace('4 days ago', 4)
df['time_from_public'] = df['time_from_public'].replace('4 weeks ago', 28)
df['time_from_public'] = df['time_from_public'].replace('5 days ago', 5)
df['time_from_public'] = df['time_from_public'].replace('6 days ago', 6)
df['time_from_public'] = df['time_from_public'].replace('3 days ago', 3)
df['time_from_public'] = df['time_from_public'].replace('1 day ago', 1)
df['time_from_public'] = df['time_from_public'].replace('2 days ago', 2)
df['time_from_public'] = df['time_from_public'].replace('6 hours ago', 0)                                                        
df['time_from_public'] = df['time_from_public'].replace('9 hours ago', 0)
df['time_from_public'] = df['time_from_public'].replace('12 minutes ago', 0)

 **$\color{blue}{\text{КОММЕНТАРИЙ}}$**

Ну ок Хотя здесь лучше сплитнуть значение столбца взять первый жлемент списка, потом проверить что у нас в троку входит - недели, часы или дни и так рассчитывать интервал

 **$\color{blue}{\text{КОММЕНТАРИЙ}}$**

Дата парсинга - 23 мая 2023 года Пожалуйста вычти из нее интервал который ты нашел чтобы получить дату публикации вакансии в привычном нам формате

In [ ]:
df = df.rename(columns = {'time_from_public': 'published_days_ago'})

### **Количество кандидатов на вакансию**

In [ ]:
df['candidates'].value_counts()

no info           100
25 applicants      26
47 applicants      22
33 applicants      21
49 applicants      20
                 ... 
87 applicants       1
143 applicants      1
35 applicants       1
59 applicants       1
178 applicants      1
Name: candidates, Length: 140, dtype: int64

In [ ]:
def extract_number(string):
    # Используем регулярное выражение для поиска числа в строке
    match = re.search(r'\d+', string)
    
    if match:
        number = int(match.group())  # Преобразуем найденное число в целочисленный формат
        return number
    
    return None  # Возвращаем None, если число не найдено

In [ ]:
# оставим в колонке только цифры
df['q_candidates'] = df['candidates'].apply(extract_number)

 **$\color{blue}{\text{КОММЕНТАРИЙ}}$**

Здорово

### **Формирование списка hard skills**

Для формирования в столбце списка хард скиллз необходимо каждый элемент описания вакансии проверить на вхождение в уже готовый список всех скиллов, и если в описании вакансии есть этот скилл - добавляем его в список.

In [ ]:
# список возможных hard skills
skills = [
    'a/b testing', 'ab testing', 'actian', 'adobe analytics', 'adobe audience manager',
    'adobe experience platform', 'adobe launch', 'adobe target', 'ai', 'airflow',
    'alooma', 'alteryx', 'amazon machine learning', 'amazon web services', 'aml',
    'amplitude', 'ansible', 'apache camel', 'apache nifi', 'apache spark',
    'api', 'asana', 'auth0', 'aws', 'aws glue', 'azure', 'azure data factory',
    'basecamp', 'bash', 'beats', 'big query', 'bigquery', 'birst', 'bitbucket',
    'blendo', 'bootstrap', 'business objects bi', 'c#', 'c++', 'caffe', 'cassandra',
    'cdata sync', 'chronograf', 'ci/cd', 'cicd', 'clickhouse', 'cloudera', 'cluvio',
    'cntk', 'cognos', 'composer', 'computer vision', 'conda', 'confluence',
    'couchbase', 'css', 'd3.js', 'dash', 'dashboard', 'data factory', 'data fusion',
    'data mining', 'data studio', 'data warehouse', 'databricks', 'dataddo',
    'dataflow', 'datahub', 'dataiku', 'datastage', 'dbconvert', 'dbeaver', 'dbt',
    'deep learning', 'dl/ml', 'docker', 'domo', 'dune', 'dv360', 'dynamodb',
    'elasticsearch', 'elt', 'erwin', 'etl', 'etleap', 'excel', 'facebook business manager',
    'fivetran', 'fuzzy', 'ga360', 'gcp', 'gensim', 'ggplot', 'git', 'github', 'gitlab',
    'google ads', 'google analytics', 'google cloud platform', 'google data flow',
    'google optimize', 'google sheets', 'google tag manager', 'google workspace',
    'grafana', 'hadoop', 'hana', 'hanagrafana', 'hbase', 'hdfs', 'hevo data', 'hightouch',
    'hive', 'hivedatabricks', 'html', 'hubspot', 'ibm coremetrics', 'inetsoft',
    'influxdb', 'informatica', 'integrate.io', 'iri voracity', 'izenda', 'java',
    'java script', 'javascript', 'jenkins', 'jira', 'jmp', 'julia', 'jupyter',
    'k2view', 'kafka', 'kantar', 'kapacitor', 'keras', 'kibana', 'kubernetes',
    'lambda', 'linux', 'logstash', 'looker', 'lstm', 'luidgi', 'matillion', 'matlab',
    'matplotlib', 'mendix', 'metabase', 'microsoft sql', 'microsoft sql server',
    'microstrategy', 'miro', 'mixpanel', 'ml', 'ml flow', 'mlflow', 'mongodb', 'mxnet',
    'mysql', 'natural nanguage processing', 'neo4j', 'nlp', 'nltk', 'nosql', 'numpy',
    'oauth', 'octave', 'omniture', 'omnituregitlab', 'openshift', 'openstack',
    'optimizely', 'oracle', 'oracle business intelligence', 'oracle data integrator',
    'pandas', 'panorama', 'pentaho', 'plotly', 'postgre', 'postgresql', 'posthog',
    'power amc', 'power bi', 'power point', 'powerbi', 'powerpivot', 'powerpoint',
    'powerquery', 'pyspark', 'python', 'pytorch', 'pytorchhevo data', 'qlik',
    'qlik sense', 'qlikview', 'querysurge', 'r', 'raphtory', 'rapidminer', 'redash',
    'redis', 'redshift', 'retool', 'rivery', 'rust', 's3', 'sa360', 'salesforce', 'sap',
    'sap business objects', 'sas', 'sas visual analytics', 'scala', 'scikit-learn',
    'scipy', 'seaborn', 'segment', 'selenium', 'sem rush', 'semrush', 'shell', 'shiny',
    'singer', 'sisense', 'skyvia', 'snowflake', 'spacy', 'spark', 'sparkml', 'splunk',
    'spotfire', 'spreadsheet', 'spss', 'sql', 'ssis', 'sssr', 'stambia', 'statistics',
    'statsbot', 'stitch', 'streamlit', 'streamsets', 'svn', 't-sql', 'tableau', 'talend',
    'targit', 'tealium', 'telegraf', 'tensorflow', 'terraapi', 'terraform', 'theano',
    'thoughtspot', 'timeseries', 'trello', 'unix', 'vba', 'vtom', 'webfocus', 'wfh',
    'xplenty', 'xtract.io', 'yellowfin', 'ga4', 'google analytics 4'
]

In [ ]:
# функция поска хард скиллов в тексте
# сначала удаляем из текста знаки препинания
# затем проверяем каждый элемент текста на вхождение в список хард скиллов
# и если вхождение есть - добавляем этот элемент в список
# по итогу возвращаем список скиллов, которые были упомянуты в тексте вакансии
def skills_f(text):
    sp_text = text.replace(',','')
    sp_text = sp_text.replace('(','')
    sp_text = sp_text.replace(')','')
    sp_text = sp_text.replace('/','')
    sp_text = sp_text.replace('.','')
    sp_text = sp_text.split()
    empty_skills = []
    for i in range(len(sp_text)):
        if sp_text[i].strip() in skills:
            empty_skills.append(sp_text[i])    
        resut = empty_skills    
    return  resut

In [ ]:
df['vac_text'] = df['vac_text'].str.lower()
df['base_skills'] = df['base_skills'].str.lower()

In [ ]:
df['skills'] = df['vac_text'].apply(skills_f)

In [ ]:
df['skills_2'] = df['base_skills'].apply(extract_skills)    

In [ ]:
# функция сложения двух списков с проверкой, не является ячейка пустой
def sum_skills(row):
        skills1 = row['skills'] if row['skills'] is not None else []
        skills2 = row['skills_2'] if row['skills_2'] is not None else []
        return skills1 + skills2
    

# Применяем функцию к датафрейму и сохраняем результат в новом столбце 'sum_skills'
df['sum_skills'] = df.apply(sum_skills, axis=1)

In [ ]:
# удаляем ненужные уже столбцы 
df = df.drop(['adr_len', 'skills', 'skills_2', 'candidates', 'Unnamed: 0', 'vac_text', 'base_skills'], axis = 1)

In [ ]:
df.head(5)

,job_title,company,adress,company_size,type_of_employment,company_sphere,employment,level,published_days_ago,city,country,q_candidates,sum_skills
0,data analyst,PharmiWeb.Jobs: Global Life Science Jobs,"Basel, Basel, Switzerland",11-50 employees,On-site,Staffing & Recruiting,Full-time,Entry level,7,Basel,Switzerland,47.0,"[sql, statistics, excel, spss, sas, sap]"
1,data analyst - logistics,Resolute Recruitment,"Coventry, England, United Kingdom",no info,On-site,no info,Full-time,Entry level,7,Coventry,United Kingdom,NaN,[]
2,data analyst - logistics,Resolute Recruitment,"Coventry, England, United Kingdom",no info,On-site,no info,Full-time,Entry level,7,Coventry,United Kingdom,NaN,[wfh]
3,data analyst (space & planning),Mole Valley Farmers,"South Molton, England, United Kingdom",no info,On-site,no info,Full-time,Entry level,7,South Molton,United Kingdom,NaN,[excel]
4,data analyst,FORFIRM,"Lugano, Ticino, Switzerland",no info,On-site,no info,Full-time,Entry level,14,Lugano,Switzerland,NaN,"[etl, gcp, aws, sql, etl, oracle, python]"


 **$\color{blue}{\text{КОММЕНТАРИЙ}}$**

Отличная работа по поиску скиллов в тексте вакансии

In [ ]:
# Разбиваем столбец 'sum_skills' на отдельные строки с помощью метода explode
df_exploded = df.explode('sum_skills')

In [ ]:
df_exploded.head(5)

,job_title,company,adress,company_size,type_of_employment,company_sphere,employment,level,published_days_ago,city,country,q_candidates,sum_skills
0,data analyst,PharmiWeb.Jobs: Global Life Science Jobs,"Basel, Basel, Switzerland",11-50 employees,On-site,Staffing & Recruiting,Full-time,Entry level,7,Basel,Switzerland,47.0,sql
0,data analyst,PharmiWeb.Jobs: Global Life Science Jobs,"Basel, Basel, Switzerland",11-50 employees,On-site,Staffing & Recruiting,Full-time,Entry level,7,Basel,Switzerland,47.0,statistics
0,data analyst,PharmiWeb.Jobs: Global Life Science Jobs,"Basel, Basel, Switzerland",11-50 employees,On-site,Staffing & Recruiting,Full-time,Entry level,7,Basel,Switzerland,47.0,excel
0,data analyst,PharmiWeb.Jobs: Global Life Science Jobs,"Basel, Basel, Switzerland",11-50 employees,On-site,Staffing & Recruiting,Full-time,Entry level,7,Basel,Switzerland,47.0,spss
0,data analyst,PharmiWeb.Jobs: Global Life Science Jobs,"Basel, Basel, Switzerland",11-50 employees,On-site,Staffing & Recruiting,Full-time,Entry level,7,Basel,Switzerland,47.0,sas


 **$\color{blue}{\text{КОММЕНТАРИЙ}}$**

Здорово

In [ ]:
# проверим образовавшийся датафрейм на дубликаты
# так как в одной вакансии искомые скилы могли встречаться несколько раз
df_exploded.duplicated().sum()

685

In [ ]:
# удалим дубликаты
df_exploded = df_exploded.drop_duplicates()

In [ ]:
df_exploded.head(10)

,job_title,company,adress,company_size,type_of_employment,company_sphere,employment,level,published_days_ago,city,country,q_candidates,sum_skills
0,data analyst,PharmiWeb.Jobs: Global Life Science Jobs,"Basel, Basel, Switzerland",11-50 employees,On-site,Staffing & Recruiting,Full-time,Entry level,7,Basel,Switzerland,47.0,sql
0,data analyst,PharmiWeb.Jobs: Global Life Science Jobs,"Basel, Basel, Switzerland",11-50 employees,On-site,Staffing & Recruiting,Full-time,Entry level,7,Basel,Switzerland,47.0,statistics
0,data analyst,PharmiWeb.Jobs: Global Life Science Jobs,"Basel, Basel, Switzerland",11-50 employees,On-site,Staffing & Recruiting,Full-time,Entry level,7,Basel,Switzerland,47.0,excel
0,data analyst,PharmiWeb.Jobs: Global Life Science Jobs,"Basel, Basel, Switzerland",11-50 employees,On-site,Staffing & Recruiting,Full-time,Entry level,7,Basel,Switzerland,47.0,spss
0,data analyst,PharmiWeb.Jobs: Global Life Science Jobs,"Basel, Basel, Switzerland",11-50 employees,On-site,Staffing & Recruiting,Full-time,Entry level,7,Basel,Switzerland,47.0,sas
0,data analyst,PharmiWeb.Jobs: Global Life Science Jobs,"Basel, Basel, Switzerland",11-50 employees,On-site,Staffing & Recruiting,Full-time,Entry level,7,Basel,Switzerland,47.0,sap
1,data analyst - logistics,Resolute Recruitment,"Coventry, England, United Kingdom",no info,On-site,no info,Full-time,Entry level,7,Coventry,United Kingdom,NaN,NaN
2,data analyst - logistics,Resolute Recruitment,"Coventry, England, United Kingdom",no info,On-site,no info,Full-time,Entry level,7,Coventry,United Kingdom,NaN,wfh
3,data analyst (space & planning),Mole Valley Farmers,"South Molton, England, United Kingdom",no info,On-site,no info,Full-time,Entry level,7,South Molton,United Kingdom,NaN,excel
4,data analyst,FORFIRM,"Lugano, Ticino, Switzerland",no info,On-site,no info,Full-time,Entry level,14,Lugano,Switzerland,NaN,etl


In [ ]:
df_exploded['sum_skills'].value_counts()

sql          162
python       126
excel        109
ai            91
tableau       75
            ... 
shell          1
asana          1
amplitude      1
lambda         1
hubspot        1
Name: sum_skills, Length: 100, dtype: int64

In [ ]:
df_exploded['sum_skills'].unique()

array(['sql', 'statistics', 'excel', 'spss', 'sas', 'sap', nan, 'wfh',
       'etl', 'gcp', 'aws', 'oracle', 'python', 'nosql', 'tableau',
       'qlikview', 'powerbi', 'dataiku', 'scala', 'spotfire',
       'spreadsheet', 'redshift', 'looker', 'api', 'r', 'bigquery',
       'jupyter', 'pandas', 'dbt', 'segment', 'ai', 'salesforce',
       'powerpoint', 'spark', 'cognos', 'jira', 'dataflow', 'airflow',
       'terraform', 'github', 'cicd', 'hadoop', 'kafka', 'talend', 'hive',
       'mongodb', 'informatica', 'alteryx', 'snowflake', 'elt',
       'metabase', 'sisense', 'panorama', 'microstrategy', 'qlik',
       'dashboard', 'numpy', 'composer', 'azure', 'javascript', 'java',
       'vba', 'bash', 'git', 'stambia', 'mysql', 'databricks',
       'confluence', 'trello', 'rapidminer', 'shell', 'html', 'css',
       'datastage', 'asana', 'amplitude', 'ssis', 'matlab', 'lambda',
       'elasticsearch', 'powerquery', 'powerpivot', 'docker', 'nlp',
       'gitlab', 'grafana', 'splunk', 'kibana

In [ ]:
req_sk = df_exploded.groupby('sum_skills')['job_title'].count(
).reset_index().sort_values(by='job_title', ascending=False)

In [ ]:
req_sk.head(20)

,sum_skills,job_title
88,sql,162
67,python,126
29,excel,109
0,ai,91
92,tableau,75
70,r,56
91,statistics,48
19,dashboard,41
63,powerbi,39
65,powerpoint,31


# **Промежуточный вывод исследования**

В ходе парсинга предоставленного файла были получены требуемеые данные по вакансиям для аналитиков данных. Данные, после проверки, готовы для визуализации в дашборде.

 **$\color{blue}{\text{ОБЩИЙ ФИДБЭК}}$**

Большое спасибо за великолепное выполнение первого этапа проекта  Ты отлично владеешь методами bs4, работаешь со строками даже без регулярных выражений - просто класс Все необходимые по ТЗ признаки выделены в отдельные столбцы Данные подготовлены к визуализации Все великолепно

Тем не менее хотелось бы дать тебе два совета
- пожалуйста проведи дополнительную обработку столбцов с городом и страной - там не всегда корректные значения
- использую интервал, который ты нашел когда была опубликована вакансия и дату парсинга рассчитай дту публикации вакансии в привычном нам виде

Если у тебя не возникло трудностей и вопросов к комментариям можно не присылать работу на повторную проверку